<a href="https://colab.research.google.com/github/mmsaad2009/olimpiadas_github/blob/main/Minha_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown
pergunta='Quando é a próxima imersão IA com o Google Gemini da Alura'

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents=pergunta
)

# Exibe a resposta na tela
display(f"Pergunta: {pergunta}")
display(Markdown(f"Resposta:\n {resposta.text}"))

'Pergunta: Quando é a próxima imersão IA com o Google Gemini da Alura'

Resposta:
 A Alura não tem uma data fixa para a próxima Imersão IA com Google Gemini. Para saber quando será a próxima edição, o ideal é:

*   **Acompanhar as redes sociais e o site da Alura:** A Alura geralmente divulga as datas das imersões em seus canais de comunicação (LinkedIn, Instagram, Twitter, site oficial).
*   **Verificar a página da Imersão IA com Google Gemini:** Procure no site da Alura pela página específica da imersão. Mesmo que não tenha uma data definida, pode ter um formulário para você se inscrever e receber notificações sobre as próximas edições.
*   **Entrar em contato com a Alura:** Se você não encontrar informações nos canais online, pode entrar em contato diretamente com a Alura através do suporte ao cliente ou canais de atendimento.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={"tools" :[{"google_search" : {}}]}
    )

# Exibe a resposta na tela
display(f"Pergunta: {pergunta}")
display("------------------------------------------------------")
display("Agora utilizando o mecanismo de busca do Google Search")
display("------------------------------------------------------")
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

display(Markdown(f"Resposta:\n {response.text}"))

'Pergunta: Quando é a próxima imersão IA com o Google Gemini da Alura'

'------------------------------------------------------'

'Agora utilizando o mecanismo de busca do Google Search'

'------------------------------------------------------'

Busca realizada: ['próxima Imersão IA com Google Gemini da Alura']
Páginas utilizadas na resposta: starten.tech



Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. O curso é gratuito e 100% online, voltado para profissionais de todas as áreas, sem necessidade de conhecimento prévio. A imersão oferece cinco aulas práticas sobre criação de prompts, automação de tarefas e desenvolvimento de agentes autônomos.

Você pode se inscrever no site da Alura.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini da Alura']
Páginas utilizadas na resposta: starten.tech



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca conteúdo no Google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é utilizar o mecanismo de busca do Google (google_search)
      para recuperar as ultimas noticias de lançamentos muito relevantes sobre o tópico abaixo.
      Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiamos das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é possivel que ele não seja tão relevante assim e pode ser subtituido por outro que tenha mais.
      Estes lançamentos devem ser atuais, de no máximo um mês antes da data de hoje.
      """
  )
  # Execução da função
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos


In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relavantes buscados você deve:
        usar a ferramenta de busca do google (google_search) para criar um plano sobre quais são os pontos mais relevantes
        que poderiamos abordar em um post sobre cada um deles. Voc~e também pode usar o (google_search) para encontrar
        mais informações sobre os temas e aprofundar.
        Ao final você ira escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar este tema
        seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
# model="gemini-2.5-pro-preview-03-25",
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
         model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o seu topico de pesquisa!")
else:
  print(f"Maravilha! Vamos então criar o posto sobre o tópico: {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n --- Resultado do Agente 1 ( Buscador ) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("---------------------------------------------------------------------")

  plano_post = agente_planejador(topico, lancamentos_buscados)
  print("\n --- Resultado do Agente 2 ( Planejador ) ---\n")
  display(to_markdown(plano_post))
  print("---------------------------------------------------------------------")

  rascunho_post = agente_redator(topico, plano_post)
  print("\n --- Resultado do Agente 3 ( Redator ) ---\n")
  display(to_markdown(rascunho_post))
  print("---------------------------------------------------------------------")

  versao_de_post = agente_revisor(topico, rascunho_post)
  print("\n --- Resultado do Agente 4 ( Revisor ) ---\n")
  display(to_markdown(versao_de_post))
  print("---------------------------------------------------------------------")



🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: charutos americanos
Maravilha! Vamos então criar o posto sobre o tópico: charutos americanos

 --- Resultado do Agente 1 ( Buscador ) ---



> Para encontrar os lançamentos de charutos americanos mais relevantes e recentes, vou usar o mecanismo de busca para identificar os charutos que geraram mais notícias e entusiasmo no último mês.
> 
> 
> Com base nas minhas buscas, aqui estão alguns dos lançamentos de charutos americanos mais recentes e relevantes, focando em notícias e entusiasmo gerado:
> 
> 1.  **Davidoff Winston Churchill Limited Edition 2025** Esta é a quinta edição da popular "Original Series". Este charuto de edição limitada apresenta uma nova mistura de tabacos da Nicarágua, República Dominicana, México e Equador, proporcionando um sabor único no formato Churchill (178 mm x 48). A produção é limitada a 18.000 caixas mundialmente. A embalagem e a faixa exibem uma pintura do icônico líder Winston Churchill.
> 2.  **VegaFina Year of the Snake 2025:** Lançado para comemorar o Ano Novo Chinês, este charuto apresenta um blend equilibrado com tabacos da República Dominicana, Nicarágua e Equador, oferecendo notas amadeiradas, florais, café e baunilha. O formato Coronas Extra (159 mm x 46) destaca-se pelo elegante "pigtail" duplo, inspirado na forma da língua da cobra.
> 3.  **Fuente Fuente OpusX La Edición de la Sociedad:** Este charuto foi lançado com uma festa em Miami e é oferecido em um único formato de 6 1/4 x 49, destacando-se pela paleta de cores azul e amarela. A marca Fuente Fuente OpusX é uma das mais procuradas pelos consumidores.
> 4.  **Compay Cigars Alma Forte:** A marca brasileira Compay Cigars apresentou o "Alma Forte", um charuto 100% nacional com tabacos cubra. Este lançamento destaca-se pela qualidade do tabaco brasileiro e pela proposta de oferecer uma experiência autêntica aos apreciadores.
> 5.  **Joya De Nicaragua Cinco De Cinco TAA Exclusive:** Este charuto foi notado como magnífico por membros de clubes de charutos.
> 
> Além desses lançamentos, é importante notar que a União Europeia está considerando tarifas sobre charutos americanos. Em abril de 2025, a União Europeia divulgou uma lista de mais de 400 produtos americanos, incluindo charutos, que poderão ser alvo de tarifas retaliatórias caso as negociações comerciais com os EUA não avancem. As tarifas adicionais podem chegar a 25%.
> 


---------------------------------------------------------------------

 --- Resultado do Agente 2 ( Planejador ) ---



> Okay, com base nos lançamentos de charutos americanos mais recentes e relevantes que você me forneceu, vou elaborar um plano de conteúdo para redes sociais. Meu objetivo é identificar os pontos mais relevantes para abordar em posts sobre cada um desses lançamentos e, ao final, escolher o tema mais interessante para um plano de postagem detalhado.
> 
> **Tópicos e Pontos Relevantes:**
> 
> 1.  **Davidoff Winston Churchill Limited Edition 2025:**
>     *   **Relevância:** Edição limitada, apelo à história e à figura icônica de Winston Churchill.
>     *   **Pontos:**
>         *   Nova mistura de tabacos de diferentes origens (Nicarágua, República Dominicana, México e Equador).
>         *   Formato Churchill (178 mm x 48).
>         *   Produção limitada (18.000 caixas mundialmente).
>         *   Design da embalagem e da faixa com pintura de Churchill.
> 2.  **VegaFina Year of the Snake 2025:**
>     *   **Relevância:** Edição comemorativa do Ano Novo Chinês, apelo à cultura e simbolismo.
>     *   **Pontos:**
>         *   Blend equilibrado com tabacos da República Dominicana, Nicarágua e Equador.
>         *   Notas de degustação: amadeirado, floral, café e baunilha.
>         *   Formato Coronas Extra (159 mm x 46) com "pigtail" duplo.
>         *   Embalagem temática do Ano da Cobra.
> 3.  **Fuente Fuente OpusX La Edición de la Sociedad:**
>     *   **Relevância:** Marca de prestígio, lançamento exclusivo em evento.
>     *   **Pontos:**
>         *   Lançamento em festa exclusiva em Miami.
>         *   Formato único de 6 1/4 x 49.
>         *   Paleta de cores azul e amarela.
>         *   Apelo à exclusividade e ao colecionismo.
> 4.  **Compay Cigars Alma Forte:**
>     *   **Relevância:** Charuto 100% nacional (brasileiro), destaque para a produção local.
>     *   **Pontos:**
>         *   Tabaco Cubra nacional.
>         *   Qualidade do tabaco brasileiro.
>         *   Proposta de experiência autêntica.
>         *   Potencial para valorizar a produção nacional.
> 5.  **Joya De Nicaragua Cinco De Cinco TAA Exclusive:**
>     *   **Relevância:** Magnífico, exclusivo.
>     *   **Pontos:**
>         *   Apreciação de membros de clubes de charutos.
> 6.  **Tarifas da União Europeia sobre charutos americanos:**
>     *   **Relevância:** Impacto no mercado, possível aumento de preços.
>     *   **Pontos:**
>         *   Ameaça de tarifas retaliatórias de até 25%.
>         *   Lista de mais de 400 produtos americanos afetados.
>         *   Impacto potencial nos preços e na disponibilidade dos charutos.
> 
> **Tema Mais Relevante e Plano de Postagem:**
> 
> Considerando a atualidade e o potencial de gerar discussões e engajamento, o tema mais relevante é o das **Tarifas da União Europeia sobre charutos americanos**.
> 
> **Plano de Postagem:**
> 
> *   **Título:** "Atenção, apreciadores de charutos americanos! A UE pode taxar seus charutos favoritos."
> *   **Assuntos a serem abordados:**
>     1.  **Introdução:**
>         *   Contextualizar a situação: a União Europeia está considerando impor tarifas sobre charutos americanos.
>         *   Explicar o motivo: negociações comerciais tensas entre os EUA e a UE.
>     2.  **Detalhes das tarifas:**
>         *   Informar sobre a porcentagem das tarifas (até 25%).
>         *   Mencionar a lista de produtos afetados (mais de 400).
>     3.  **Impacto para os consumidores:**
>         *   Possível aumento de preços dos charutos americanos na Europa.
>         *   Potencial impacto na disponibilidade de certas marcas e edições limitadas.
>     4.  **Call to action:**
>         *   Incentivar os seguidores a expressarem suas opiniões sobre o assunto.
>         *   Perguntar se eles estão preocupados com o possível aumento de preços.
>         *   Sugerir que entrem em contato com seus representantes políticos para expressar sua preocupação.
>     5.  **Hashtags:**
>         *   #charutosamericanos #tarifasUE #comérciointernacional #cigars #charuto #tabaco #notícias #economia #preços
> 
> **Próximos Passos:**
> 
> Com este plano em mãos, o próximo passo seria desenvolver o texto completo do post, buscando fontes adicionais para confirmar as informações e enriquecer o conteúdo. Também seria importante criar um visual atraente para o post, como uma imagem ou vídeo relacionado ao tema.
> 


---------------------------------------------------------------------

 --- Resultado do Agente 3 ( Redator ) ---



> 🚨 Atenção, amantes de charutos americanos! 🚨
> 
> Uma notícia que pode impactar diretamente o seu próximo charuto: a União Europeia está considerando impor tarifas de até 25% sobre charutos americanos! 😱
> 
> Isso acontece devido a tensões nas negociações comerciais entre os EUA e a UE, e a lista de produtos afetados é gigante, com mais de 400 itens! 😬
> 
> **E o que isso significa pra você?** 
> 📈 Aumento nos preços dos seus charutos favoritos.
> 😕 Possível diminuição na disponibilidade de algumas marcas e edições limitadas.
> 
> Queremos saber a sua opinião! 🤔
> 
> Você está preocupado com essa possível taxação? Acha que isso vai impactar o seu consumo de charutos americanos? Deixe seu comentário! 👇
> 
> E não se esqueça: sua voz importa! Entre em contato com seus representantes políticos e mostre sua preocupação! 🗣️
> 
> #charutosamericanos #tarifasUE #comérciointernacional #charuto
> 


---------------------------------------------------------------------

 --- Resultado do Agente 4 ( Revisor ) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais interessante e engajador para o público do Instagram, com foco em jovens de 18 a 30 anos. Aqui estão minhas sugestões:
> 
> **Problemas identificados e sugestões:**
> 
> *   **Tom:** O tom está um pouco alarmista ("Atenção!", "😱"). Podemos suavizar para algo mais informativo e interessante.
> *   **Emojis:** O uso excessivo de emojis pode cansar. Vamos equilibrar.
> *   **Chamada para ação:** Podemos deixar a chamada para ação mais direta e convidativa.
> *   **Hashtags:** Adicionar algumas hashtags mais específicas e relevantes para o público.
> 
> **Rascunho revisado:**
> 
> E aí, galera do charuto! 💨🇺🇸
> 
> Notícia importante pra quem curte um bom charuto americano: a União Europeia pode taxar esses produtos em até 25%! 💸
> 
> **Por que isso tá rolando?**
> A parada é que EUA e UE estão meio que "trocando farpas" nas negociações de comércio, e adivinha quem pode pagar o pato? Nossos charutos! 🥺 São mais de 400 produtos na lista!
> 
> **E o que muda pra gente?**
> 💰 Se preparem para preços mais altos nos seus charutos favoritos.
> Limitadas.
> 🤔 Talvez role até um sumiço de algumas marcas e edições
> 
> **E aí, qual a boa?**
> Queremos saber o que você pensa! Essa possível taxação te preocupa? Deixe seu comentário! 👇
> 
> Sua opinião faz a diferença! Se liga nos seus representantes políticos e mostra a sua voz! 🗣️
> 
> #charutosamericanos #taxasUE #charutando #cigaraficionado #charutolife #fumetas #confrariadocharuto #cigar #charuto
> 
> **Justificativas das mudanças:**
> 
> *   **Tom:** Mais casual e informativo, usando gírias como "galera", "parada" e "e aí, qual a boa?".
> *   **Emojis:** Uso mais estratégico para complementar o texto, sem exagerar.
> *   **Chamada para ação:** Mais direta ("Deixe seu comentário!").
> *   **Hashtags:** Adicionadas hashtags mais específicas para o nicho de charutos e para o público jovem.
> *   **Formatação:** Quebra de linhas e uso de negrito para facilitar a leitura no mobile.


---------------------------------------------------------------------
